In [44]:
# Basic Libraries
import os
import shutil
import random
import numpy as np
import pandas as pd

# Image Processing
import cv2
from PIL import Image, ImageEnhance
from skimage.util import random_noise

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Scikit-learn for Model Preparation
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

# Additional Libraries for Image Handling and File Operations
import glob
from glob import glob
import matplotlib.image as mpimg
import pydicom

# TensorFlow and Keras for Deep Learning
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model, load_model, Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Attention, Add, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import ModelCheckpoint

print("Imports Complete")

Imports Complete


In [45]:
# Path to the input and output directories
train_input_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/'
test_input_path= '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'

df_train = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv")
df_train_series_descriptions = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv")
df_label_coord = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv")
df_test_series_descriptions = pd.read_csv("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv")

# Output Paths
output_path_train_dir = '/kaggle/working/train_images'
output_path_augmented_dir ='/kaggle/working/augmented_images'

In [46]:
# Create image paths
df_label_coord['image_path'] = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/" + \
                               df_label_coord['study_id'].astype(str) + "/" + \
                               df_label_coord['series_id'].astype(str) + "/" + \
                               df_label_coord['instance_number'].astype(str) + ".dcm"

# Melt the df_train DataFrame
df_train_melted = df_train.melt(id_vars=['study_id'], var_name='condition_level', value_name='severity')

# Split 'condition_level' to extract 'condition' and 'level'
df_train_melted[['conditions', 'level']] = df_train_melted['condition_level'].str.rsplit('_', n=2, expand=True).iloc[:, 1:]
df_train_melted['condition'] = df_train_melted['condition_level'].apply(lambda x: '_'.join(x.split('_')[:-2])).str.replace("_", " ").str.title()
df_train_melted['level'] = df_train_melted['conditions'].str.upper() + "/" + df_train_melted['level'].str.upper()

# Drop the original 'condition_level' column
df_train_melted = df_train_melted.drop(columns=['condition_level', 'conditions'])

# Merge DataFrames on 'study_id', 'level', and 'condition'
df_final = pd.merge(df_label_coord, df_train_melted, on=['study_id', 'level', 'condition'], how='inner')

# Ensure the 'series_description' column exists before trying to reorder
if 'series_description' in df_train_series_descriptions.columns:
    # Merge df_final with df_train_series_descriptions on 'study_id' and 'series_id'
    df_final_filtered = pd.merge(df_final, df_train_series_descriptions[['study_id', 'series_id', 'series_description']],
                                 on=['study_id', 'series_id'], how='left')

    # Reorder columns to place 'series_description' immediately after 'series_id'
    columns_order = ['study_id', 'series_id', 'series_description', 'instance_number', 'condition', 'level', 'x', 'y', 'image_path', 'severity']
    
    # Ensure that 'series_description' exists in the DataFrame before reordering columns
    if 'series_description' in df_final_filtered.columns:
        df_final_filtered = df_final_filtered[columns_order]
    else:
        print("Warning: 'series_description' column not found after merging.")
else:
    print("Warning: 'series_description' column not found in the input data.")
    
df_final_filtered.head()

,study_id,series_id,series_description,instance_number,condition,level,x,y,image_path,severity
0,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild
1,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild
2,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild
3,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild
4,4003253,702807833,Sagittal T2/STIR,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild


In [47]:
# Create the row_id column
df_final_filtered['row_id'] = (
    df_final_filtered['study_id'].astype(str) + '_' +
    df_final_filtered['condition'].str.lower().str.replace(' ', '_') + '_' +
    df_final_filtered['level'].str.lower().str.replace('/', '_')
)

df_final_filtered.sample()

,study_id,series_id,series_description,instance_number,condition,level,x,y,image_path,severity,row_id
4899,425970461,201607168,Axial T2,12,Right Subarticular Stenosis,L3/L4,164.71137,194.798834,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,Normal/Mild,425970461_right_subarticular_stenosis_l3_l4


In [48]:
# Function to convert DICOM pixel array to PNG
def readdcm_writepng_image(src_dicom_pixelarray, dest_path_png):
    src_dicom_pixelarray = np.array(src_dicom_pixelarray)
    standardized_image_data = ((src_dicom_pixelarray - src_dicom_pixelarray.min()) / 
                               (src_dicom_pixelarray.max() - src_dicom_pixelarray.min() + 1e-10)) * 255
    standardized_image_data = standardized_image_data.astype(np.uint8)
    final_image_to_png = cv2.resize(standardized_image_data, (320, 320), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(dest_path_png, final_image_to_png)

# Remove previous output directory for fresh writing
if os.path.isdir(output_path_train_dir):
    shutil.rmtree(output_path_train_dir)

# Drop duplicates based on 'image_path' to ensure each image is converted only once
unique_images_df = df_final_filtered.drop_duplicates(subset='image_path')

# Create a new DataFrame to store paths to the converted images
df_train_images_png = pd.DataFrame(columns=df_final_filtered.columns)

# Convert only unique labeled images
for index, row in tqdm(unique_images_df.iterrows(), total=len(unique_images_df)):
    study_id = row['study_id']
    # Apply the replacement to series_description
    series_description = row['series_description'].replace(' ', '_').replace('/', '_')
    instance_number = row['instance_number']
    
    # Construct the destination path for the PNG file
    dest_path = f'{output_path_train_dir}/{study_id}/{series_description}/{instance_number}.png'
    
    # Ensure directory exists
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)
    
    # Read the DICOM image and convert it to PNG
    dicom_image = pydicom.dcmread(row['image_path'])
    readdcm_writepng_image(dicom_image.pixel_array, dest_path)
    
    # Copy the row and update the image path to the new PNG path
    new_row = row.copy()
    new_row['image_path'] = dest_path
    
    # Replace series_description in the new_row DataFrame
    new_row['series_description'] = series_description
    
    # Append the new row to the new DataFrame using pd.concat
    df_train_images_png = pd.concat([df_train_images_png, pd.DataFrame([new_row])], ignore_index=True)

print("Conversion to PNG completed.")

# Save the new DataFrame to a CSV file (optional)
df_train_images_png.to_csv('/kaggle/working/df_png_paths.csv', index=False)

print("Dataframe saved")
df_train_images_png.sample()

  0%|          | 0/24546 [00:00<?, ?it/s]/tmp/ipykernel_36/4124105440.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_train_images_png = pd.concat([df_train_images_png, pd.DataFrame([new_row])], ignore_index=True)
 30%|███       | 7465/24546 [02:26<05:34, 51.12it/s]


KeyboardInterrupt: 

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from albumentations import (
    HorizontalFlip, VerticalFlip, Rotate, RandomBrightnessContrast,
    ColorJitter, GridDistortion, RandomGamma, GaussNoise, Compose,
    CLAHE, Solarize, Posterize, ShiftScaleRotate, ElasticTransform,
    ToGray, HueSaturationValue
)

# Step 1: Initialise Paths
df_converted_data = pd.read_csv("/kaggle/working/df_png_paths.csv")
output_images_dir = '/kaggle/working/augmented_images'
csv_output_path = '/kaggle/working/df_augmented_final.csv'

# Ensure output directory exists
os.makedirs(output_images_dir, exist_ok=True)

# Step 2: Assume df_png_paths is already defined with the necessary data
# You need to format the series_description
df_augmented = df_train_images_png.copy()
df_augmented['series_description'] = df_augmented['series_description'].str.replace(r'[ /]', '_', regex=True)

# Step 3: Define color map augmentation functions
def apply_color_map(image, colormap):
    return cv2.applyColorMap(image, colormap)

# Step 4: Define augmentation techniques
albumentations_augmentations = [
    Compose([Rotate(limit=90), HorizontalFlip()]),
    Compose([Rotate(limit=180)]),
    Compose([Rotate(limit=270), HorizontalFlip()]),
    Compose([ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)]),
    Compose([GaussNoise(), VerticalFlip()]),
    Compose([GridDistortion()]),
    Compose([ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15)]),
    Compose([ElasticTransform(alpha=1, sigma=50, alpha_affine=None)]),  # Updated line
    Compose([CLAHE(), HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20)]),
    Compose([Solarize(threshold=128.0), Posterize(num_bits=4)]),
    Compose([ToGray()])
]

color_map_augmentations = [
    (cv2.COLORMAP_VIRIDIS, 'viridis'),
    (cv2.COLORMAP_PLASMA, 'plasma'),
    (cv2.COLORMAP_INFERNO, 'inferno'),
    (cv2.COLORMAP_MAGMA, 'magma'),
]

# Combine all augmentations into one list
all_augmentations = albumentations_augmentations + color_map_augmentations

# Define how many times to augment each image for Moderate and Severe classes
num_augmentations_per_image_severe = 10 # Augment each 'Severe' image 6 times
num_augmentations_per_image_moderate = 4  # Augment each 'Moderate' image 1 time

def augment_image(row):
    image_path = row['image_path']
    image = cv2.imread(image_path)  # Load the image using OpenCV

    # Check if the image was loaded successfully
    if image is None:
        print(f"Warning: Unable to load image at path: {image_path}")
        return []  # Return an empty list if the image could not be loaded

    coords = [row['x'], row['y']]  # Extract coordinates
    augmented_images = []  # Store augmented images for this row
    image_height, image_width = image.shape[:2]

    # Determine the number of augmentations based on severity
    if row['severity'] == 'Severe':
        num_augmentations = num_augmentations_per_image_severe
    elif row['severity'] == 'Moderate':
        num_augmentations = num_augmentations_per_image_moderate
    else:
        return []  # Skip if severity is not 'Moderate' or 'Severe'

    for _ in range(num_augmentations):
        # Choose an augmentation
        aug_index = np.random.choice(len(all_augmentations))
        aug = all_augmentations[aug_index]

        try:
            if isinstance(aug, tuple):
                # Apply the color map augmentation
                colormap, name = aug
                image_aug = apply_color_map(image, colormap)
                aug_name = name  # Use the color map name directly
            else:
                # Apply the Albumentations augmentation
                augmented = aug(image=image)
                image_aug = augmented['image']

                # Get the augmentation names
                aug_name = '_'.join([type(t).__name__ for t in aug.transforms])

                # Update coordinates based on the applied transformations
                for t in aug.transforms:
                    if isinstance(t, HorizontalFlip):
                        coords[0] = image_width - coords[0]
                    if isinstance(t, VerticalFlip):
                        coords[1] = image_height - coords[1]
                    if isinstance(t, Rotate):
                        angle = t.limit if isinstance(t.limit, (int, float)) else t.limit[1]
                        if angle == 90:
                            coords = [coords[1], image_width - coords[0]]
                        elif angle == 180:
                            coords = [image_width - coords[0], image_height - coords[1]]
                        elif angle == 270:
                            coords = [image_height - coords[1], coords[0]]

            # Create subfolder structure
            study_id = row['study_id']
            series_id = row['series_id']
            series_description = row['series_description'].replace(' ', '_')  # Replace spaces with underscores
            output_subfolder = os.path.join(output_images_dir, str(study_id), series_description)
            os.makedirs(output_subfolder, exist_ok=True)

            # Generate new file name with the augmentation name and instance number
            instance_number = row['instance_number']
            augmented_image_path = os.path.join(output_subfolder, f"{aug_name}_{instance_number}.png")

            # Save the augmented image
            cv2.imwrite(augmented_image_path, image_aug)

            augmented_images.append({
                'study_id': study_id,
                'series_id': series_id,
                'series_description': series_description,
                'instance_number': instance_number,
                'x': coords[0],
                'y': coords[1],
                'condition': row['condition'],
                'level': row['level'],
                'image_path': augmented_image_path,
                'severity': row['severity']
            })
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")

    return augmented_images


# Step 7: Filter only Moderate and Severe classes for augmentation
df_filtered = df_augmented[df_augmented['severity'].isin(['Moderate', 'Severe'])]

# Step 8: Use parallel processing to augment images
augmented_data = []

with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(augment_image, [row for _, row in df_filtered.iterrows()]), total=len(df_filtered)))

# Flatten the results and filter out None values
augmented_data = [item for sublist in results for item in sublist if item is not None]

# Step 9: Collect the results into a DataFrame
df_augmented_final = pd.DataFrame(augmented_data)

# Save the augmented DataFrame to a CSV file
df_augmented_final.to_csv(csv_output_path, index=False)

print(f"Total processed images: {len(augmented_data)}")

In [ ]:
# read data
train_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/'
train_desc  = pd.read_csv(train_path + 'train_series_descriptions.csv')
test_desc   = pd.read_csv(train_path + 'test_series_descriptions.csv')
test_desc['series_description'] = test_desc['series_description'].str.replace(r'[ /]', '_', regex=True)
test_desc.head(5)

In [ ]:
from glob import glob

output_path = '/kaggle/working/test_images/'

# Function to convert DICOM pixel array to PNG
def readdcm_writepng_image(src_dicom_pixelarray, dest_path_png):
    src_dicom_pixelarray = np.array(src_dicom_pixelarray)
    standardized_image_data = ((src_dicom_pixelarray - src_dicom_pixelarray.min()) / 
                               (src_dicom_pixelarray.max() - src_dicom_pixelarray.min() + 1e-10)) * 255
    standardized_image_data = standardized_image_data.astype(np.uint8)
    final_image_to_png = cv2.resize(standardized_image_data, (320, 320), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(dest_path_png, final_image_to_png)

# Remove previous output directory for fresh writing
if os.path.isdir(output_path):
    shutil.rmtree(output_path)


# Iterate over the test data
for idx, row in tqdm(test_desc.iterrows(), total=len(test_desc)):
    study_id = row['study_id']
    series_id = row['series_id']
    series_desc = row['series_description'].replace(' ', '_').replace('/', '_')
    
    # Define the new directory structure for PNGs
    series_output_dir = f'{output_path}/{study_id}/{series_desc}'
    os.makedirs(series_output_dir, exist_ok=True)
    
    # Get all DICOM files in this series
    series_dicom_dir = f'{test_input_path}/{study_id}/{series_id}'
    dicom_files = glob(f'{series_dicom_dir}/*.dcm')
    
    # Convert each DICOM file to PNG
    for dicom_file in dicom_files:
        dicom_image = pydicom.dcmread(dicom_file)
        image_filename = os.path.splitext(os.path.basename(dicom_file))[0]  # Use SOPInstanceUID for naming
        image_dicom_pixelarray = dicom_image.pixel_array
        
        dest_path = f'{series_output_dir}/{image_filename}.png'
        readdcm_writepng_image(image_dicom_pixelarray, dest_path)

In [ ]:
# Define the base path for test images
base_path = '/kaggle/working/test_images'

# Function to get image paths for a series
def get_image_paths(row):
    series_path = os.path.join(base_path, str(row['study_id']), str(row['series_description']))
    if os.path.exists(series_path):
        return [os.path.join(series_path, f) for f in os.listdir(series_path) if os.path.isfile(os.path.join(series_path, f))]
    return []

# Mapping of series_description to conditions
condition_mapping = {
    'Sagittal_T1': {'left': 'left_neural_foraminal_narrowing', 'right': 'right_neural_foraminal_narrowing'},
    'Axial_T2': {'left': 'left_subarticular_stenosis', 'right': 'right_subarticular_stenosis'},
    'Sagittal_T2_STIR': 'spinal_canal_stenosis'
}

# Create a list to store the expanded rows
expanded_rows = []

# Expand the dataframe by adding new rows for each file path
for index, row in test_desc.iterrows():
    image_paths = get_image_paths(row)
    conditions = condition_mapping.get(row['series_description'], {})
    if isinstance(conditions, str):  # Single condition
        conditions = {'left': conditions, 'right': conditions}
    for side, condition in conditions.items():
        for image_path in image_paths:
            expanded_rows.append({
                'study_id': row['study_id'],
                'series_id': row['series_id'],
                'series_description': row['series_description'],
                'image_path': image_path,
                'condition': condition,
                'row_id': f"{row['study_id']}_{condition}"
            })

# Create a new dataframe from the expanded rows
expanded_test_desc = pd.DataFrame(expanded_rows)

# Extracting the instance number from the image_path
expanded_test_desc['instance_number'] = expanded_test_desc['image_path'].apply(
    lambda x: int(os.path.splitext(os.path.basename(x))[0])  # Get the filename without extension
)

levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']

# Function to update row_id with levels
def update_row_id(row, levels):
    level = levels[row.name % len(levels)]
    return f"{row['study_id']}_{row['condition']}_{level}"

# Update row_id in expanded_test_desc to include levels
expanded_test_desc['row_id'] = expanded_test_desc.apply(lambda row: update_row_id(row, levels), axis=1)

# Display the resulting dataframe
expanded_test_desc.head(5)

In [ ]:
# Drop rows with severity equal to 0 or NaN
df_final_filtered_cleaned = df_train_images_png[(df_train_images_png['severity'] != 0) & (df_train_images_png['severity'].notna())]
df_augmented_cleaned = df_augmented_final[(df_augmented_final['severity'] != 0) & (df_augmented_final['severity'].notna())]

# Display the resulting DataFrame
print(f"Data after removing rows with severity 0 or NaN: {df_final_filtered_cleaned.shape[0]} samples")
print(f"Data after removing rows with severity 0 or NaN: {df_augmented_cleaned.shape[0]} samples")

# Concatenate the cleaned DataFrames
df_concat = pd.concat([df_final_filtered_cleaned, df_augmented_cleaned], ignore_index=True)

# Check the class distribution after balancing
print(df_concat["severity"].value_counts())

In [ ]:
# List to store paths of corrupted files
corrupted_files = []

# Check each image in the dataset
for index, row in df_concat.iterrows():
    img_path = row['image_path']
    try:
        # Try to open the image file
        img = Image.open(img_path)
        img.verify()  # Verify that it is a valid image
    except (IOError, SyntaxError) as e:
        corrupted_files.append(img_path)

# Remove corrupted files from the DataFrame
df_concat_cleaned = df_concat[~df_concat['image_path'].isin(corrupted_files)]

# Create the final augmented DataFrame with cleaned data
df_concat = df_concat_cleaned.copy()

# Print the number of corrupted files found and removed
print(f"Number of corrupted files removed: {len(corrupted_files)}")

# Print the number of valid rows in the final DataFrame
print(f"Number of valid rows in the final DataFrame: {df_concat.shape[0]}")

In [ ]:
# Assuming df_dataset is your original DataFrame with columns: 
# 'series_description', 'condition', 'level', 'severity', and others.

# Function to perform oversampling on a full DataFrame while considering series_description and severity
def oversample_by_severity(df):
    # Create the feature set (X) and target (y)
    X = df.drop(columns=['severity'])  # Keep all other columns except 'severity'
    y = df['severity']  # Target is the 'severity' column
    
    # Apply RandomOverSampler based on the severity imbalance
    ros = RandomOverSampler(sampling_strategy='auto')  # Automatically balance all severity classes
    X_resampled, y_resampled = ros.fit_resample(X, y)
    
    # Combine resampled X and y back into a DataFrame
    df_resampled = pd.DataFrame(X_resampled, columns=X.columns)  # Features
    df_resampled['severity'] = y_resampled  # Add back the severity column
    
    return df_resampled

# Initialize an empty list to store oversampled data
oversampled_dfs = []

# Iterate over each series_description group
for series_description, group_df in df_concat.groupby('series_description'):
    print(f"Oversampling for series_description: {series_description}")
    
    # Perform oversampling for the current group based on severity
    df_resampled = oversample_by_severity(group_df)
    
    # Add back the series_description column to the resampled DataFrame
    df_resampled['series_description'] = series_description
    
    # Append the resampled DataFrame to the list
    oversampled_dfs.append(df_resampled)

# Concatenate all oversampled DataFrames into one
df_oversampled = pd.concat(oversampled_dfs, ignore_index=True)

# Now df_oversampled contains the oversampled data, balanced by severity for each series_description
print(f"Oversampled dataset size: {df_oversampled.shape}")


In [52]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your CNN model with customizable parameters for each convolutional layer
def create_cnn_model(input_shape, num_classes, dropout_rate=0.25, learning_rate=0.0001, 
                     use_batch_norm=False, use_attention=False, 
                     conv_filters=[32, 64, 128], dense_units=128):
    model = models.Sequential()
    
    # First convolutional block
    model.add(layers.Conv2D(conv_filters[0], (3, 3), activation='relu', input_shape=input_shape))
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(dropout_rate))

    # Second convolutional block
    model.add(layers.Conv2D(conv_filters[1], (3, 3), activation='relu'))
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(dropout_rate))

    # Third convolutional block
    model.add(layers.Conv2D(conv_filters[2], (3, 3), activation='relu'))
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(dropout_rate))

    # Attention Layer (example implementation)
    if use_attention:
        model.add(layers.GlobalAveragePooling2D())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(num_classes, activation='softmax'))

    # Fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(dense_units, activation='relu'))
    if use_batch_norm:
        model.add(layers.BatchNormalization())

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    # Compile the model with a specified optimizer
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])    
    return model

# Set parameters
image_size = (320, 320)  # Target size for images
batch_size = 64
epochs = 10
input_shape = (*image_size, 3)  # Assuming RGB images
num_classes = 3  # Severity has 3 categories

# Define custom hyperparameters for each series
hyperparameters = {
    'Series_1': {'dropout_rate': 0.2, 'learning_rate': 0.0001, 'use_batch_norm': True, 'use_attention': True, 'conv_filters': [32, 64, 128], 'dense_units': 128},
    'Series_2': {'dropout_rate': 0.3, 'learning_rate': 0.0005, 'use_batch_norm': False, 'use_attention': True, 'conv_filters': [64, 128, 64], 'dense_units': 64},
    'Series_3': {'dropout_rate': 0.4, 'learning_rate': 0.001, 'use_batch_norm': True, 'use_attention': False, 'conv_filters': [128, 64, 32], 'dense_units': 256},
    # Add more series and their hyperparameters as needed
}

# Assuming df_dataset is your DataFrame containing the dataset with 'severity' and 'series_description'

# Get unique series_descriptions from the DataFrame
series_descriptions = df_oversampled['series_description'].unique()

# Iterate through each series_description
for series in series_descriptions:
    print(f"\nTraining model for series description: {series}")
    
    # Filter the DataFrame for the current series_description
    series_df = df_oversampled[df_oversampled['series_description'] == series]
    
    # Ensure we have 3 classes in the 'severity' column
    assert series_df['severity'].nunique() == 3, "The 'severity' column must have exactly 3 classes"
    
    # Split the data into train, validation, and test sets
    train_df, test_df = train_test_split(series_df, test_size=0.25, stratify=series_df['severity'])
    train_df, val_df = train_test_split(train_df, test_size=0.35, stratify=train_df['severity'])

    # ImageDataGenerator setup
    datagen = ImageDataGenerator(rescale=1.0/255)

    # Create ImageDataGenerators for train, validation, and test sets
    train_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='image_path',
        y_col='severity',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    val_generator = datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col='image_path',
        y_col='severity',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    test_generator = datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='image_path',
        y_col='severity',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    # Get hyperparameters for the current series
    series_hyperparams = hyperparameters.get(series, {'dropout_rate': 0.25, 'learning_rate': 0.0001, 'use_batch_norm': False, 'use_attention': False, 'conv_filters': [32, 64, 128], 'dense_units': 128})
    
    # Create CNN model for the current series_description with custom hyperparameters
    model = create_cnn_model(input_shape, num_classes, 
                             dropout_rate=series_hyperparams['dropout_rate'], 
                             learning_rate=series_hyperparams['learning_rate'],
                             use_batch_norm=series_hyperparams['use_batch_norm'],
                             use_attention=series_hyperparams['use_attention'],
                             conv_filters=series_hyperparams['conv_filters'],
                             dense_units=series_hyperparams['dense_units'])

    # Train the model
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        verbose=1
    )

    # Save the model after training
    model.save(f"/kaggle/working/{series}_model.h5")

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(test_generator)
    print(f"Test Accuracy for {series}: {test_acc:.4f}")


Training model for series description: Axial_T2
Found 14657 validated image filenames belonging to 3 classes.
Found 7925 validated image filenames belonging to 3 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 6369 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 3397 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(


Found 7589 validated image filenames belonging to 3 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 3194 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


...

114/114 ━━━━━━━━━━━━━━━━━━━━ 73s 582ms/step - accuracy: 0.4278 - loss: 1.1893 - val_accuracy: 0.4835 - val_loss: 1.0055
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 60s 504ms/step - accuracy: 0.4800 - loss: 0.9477 - val_accuracy: 0.4910 - val_loss: 0.9642
Epoch 3/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 60s 503ms/step - accuracy: 0.5198 - loss: 0.9116 - val_accuracy: 0.5212 - val_loss: 0.9538
Epoch 4/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 60s 504ms/step - accuracy: 0.5504 - loss: 0.8799 - val_accuracy: 0.5651 - val_loss: 0.9213
Epoch 5/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 61s 512ms/step - accuracy: 0.5945 - loss: 0.8372 - val_accuracy: 0.6012 - val_loss: 0.8763
Epoch 6/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 60s 506ms/step - accuracy: 0.6436 - loss: 0.7613 - val_accuracy: 0.6415 - val_loss: 0.8566
Epoch 7/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 60s 507ms/step - accuracy: 0.6876 - loss: 0.6964 - val_accuracy: 0.6444 - val_loss: 0.8063
Epoch 8/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 60s 506ms/step - accuracy: 0.7228 - loss: 0.6372 - val

/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 991 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 540 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 500 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


37/37 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.4049 - loss: 1.4835 - val_accuracy: 0.5080 - val_loss: 1.0036
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 498ms/step - accuracy: 0.5171 - loss: 0.9260 - val_accuracy: 0.5406 - val_loss: 0.9788
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 497ms/step - accuracy: 0.5687 - loss: 0.8956 - val_accuracy: 0.6290 - val_loss: 0.9355
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 501ms/step - accuracy: 0.6238 - loss: 0.8320 - val_accuracy: 0.6059 - val_loss: 0.9887
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 499ms/step - accuracy: 0.6694 - loss: 0.7677 - val_accuracy: 0.5518 - val_loss: 0.9359
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 497ms/step - accuracy: 0.6864 - loss: 0.7558 - val_accuracy: 0.7150 - val_loss: 0.8212
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 499ms/step - accuracy: 0.7261 - loss: 0.6646 - val_accuracy: 0.7253 - val_loss: 0.8248
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 491ms/step - accuracy: 0.7837 - loss: 0.5807 - val_accuracy: 0.6553 -

In [55]:
# Assuming 'expanded_test_desc' already exists and contains the image paths and row IDs
# Create a directory for the models
models_directory = '/kaggle/working'  # Update with your actual models directory

# Initialize an empty list to collect predictions
submission_data = []

# Get unique series descriptions from the existing DataFrame
series_descriptions = expanded_test_desc['series_description'].unique()


# Loop through each unique series description
for series in series_descriptions:
    print(f"\nProcessing series: {series}")
    
    # Load the corresponding model
    model_path = os.path.join(models_directory, f"{series}_model.h5")
    
    try:
        model = load_model(model_path)
    except Exception as e:
        print(f"Error loading model for {series}: {e}")
        continue
    
    # Filter the DataFrame for the current series
    series_df = expanded_test_desc[expanded_test_desc['series_description'] == series]
    
    # Create a test data generator
    test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=expanded_test_desc,
        x_col='image_path',
        y_col=None,  # No labels for test data
        class_mode=None,
        target_size=(320, 320),  # Adjust to your model's expected input size
        batch_size=32,
        shuffle=False,
        seed=42
    )
    
    # Make predictions
    predictions = model.predict(test_generator, verbose=1)

    # Convert predictions to probabilities
    for idx, row in enumerate(series_df.itertuples()):
        row_id = row.row_id
        pred = predictions[idx]

        # Assuming the model outputs class scores for normal/mild, moderate, and severe
        normal_mild_prob = pred[0]  # Replace with appropriate index if necessary
        moderate_prob = pred[1]      # Replace with appropriate index if necessary
        severe_prob = pred[2]        # Replace with appropriate index if necessary

        # Append the results to the submission data
        submission_data.append({
            'row_id': row_id,
            'normal_mild': normal_mild_prob,
            'moderate': moderate_prob,
            'severe': severe_prob
        })

# Create a DataFrame for submission
submission_df = pd.DataFrame(submission_data)


# Set the display format for floating-point numbers to show decimals
pd.options.display.float_format = '{:.8f}'.format  # Change the number of decimal places as needed


# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

print("Submission DataFrame created successfully.")



Processing series: Sagittal_T1
Found 194 validated image filenames.
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step

Processing series: Axial_T2
Found 194 validated image filenames.
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step

Processing series: Sagittal_T2_STIR
Found 194 validated image filenames.
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step
Submission DataFrame created successfully.


In [56]:
expanded_test_desc["row_id"].value_counts()

row_id
44036939_spinal_canal_stenosis_l4_l5               10
44036939_spinal_canal_stenosis_l5_s1               10
44036939_spinal_canal_stenosis_l3_l4               10
44036939_spinal_canal_stenosis_l2_l3               10
44036939_spinal_canal_stenosis_l1_l2               10
44036939_right_subarticular_stenosis_l4_l5         10
44036939_right_subarticular_stenosis_l3_l4         10
44036939_left_subarticular_stenosis_l2_l3          10
44036939_left_subarticular_stenosis_l1_l2          10
44036939_right_subarticular_stenosis_l5_s1          9
44036939_left_subarticular_stenosis_l4_l5           9
44036939_left_subarticular_stenosis_l3_l4           9
44036939_left_subarticular_stenosis_l5_s1           9
44036939_right_subarticular_stenosis_l1_l2          9
44036939_right_subarticular_stenosis_l2_l3          9
44036939_right_neural_foraminal_narrowing_l4_l5     5
44036939_left_neural_foraminal_narrowing_l2_l3      5
44036939_left_neural_foraminal_narrowing_l1_l2      5
44036939_right_neural

In [57]:
# Count occurrences of each unique row_id
row_id_counts = submission_df['row_id'].value_counts()

# Group by 'row_id' and calculate the mean for each group
mean_df = submission_df.groupby('row_id').mean().reset_index()

# Display the mean DataFrame
mean_df

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.65952271,0.12109806,0.21937919
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.47745237,0.05419617,0.46835145
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.57662284,0.18902835,0.23434874
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.52638257,0.12610979,0.34750766
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.44783497,0.13753089,0.41463414
5,44036939_left_subarticular_stenosis_l1_l2,0.88829786,0.00184982,0.10985237
6,44036939_left_subarticular_stenosis_l2_l3,0.89754641,0.00471216,0.09774145
7,44036939_left_subarticular_stenosis_l3_l4,0.85379273,0.00334296,0.14286438
8,44036939_left_subarticular_stenosis_l4_l5,0.96247536,0.00059654,0.03692815
9,44036939_left_subarticular_stenosis_l5_s1,0.90697563,0.00394123,0.08908312
